# Инициализация необходимых библиотек

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.model_selection import iterative_train_test_split
import numpy as np
import pandas as pd
import joblib
from catboost import CatBoostClassifier

# Загрузка данных для обучения модели

In [38]:
# Загрузка данных
df = pd.read_csv("data/Готовые данные.csv")

# Просмотр первых строк и структуры данных
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812 entries, 0 to 1811
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   comment                             1812 non-null   object 
 1   id                                  1812 non-null   float64
 2   rating                              1812 non-null   float64
 3   Вопрос решен                        1812 non-null   int64  
 4   Качество выполненной работы         1812 non-null   int64  
 5   Компетентность работника в общение  1812 non-null   int64  
 6   Не удовлетворен                     1812 non-null   int64  
 7   Скорость выполненной работы         1812 non-null   int64  
 8   Не понравился результат             1812 non-null   int64  
 9   Не понравилось качество             1812 non-null   int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 141.7+ KB


(                                        comment         id  rating  \
 0                                       спасибо  2945792.0     5.0   
 1                                      спасибо!  3234340.0     5.0   
 2                                      Отлично   3380332.0     5.0   
 3  Благодарю за оперативное решение проблемы !   3381812.0     5.0   
 4         Прекрасный специалист! Побольше таких  3461991.0     5.0   
 
    Вопрос решен  Качество выполненной работы  \
 0             1                            0   
 1             1                            0   
 2             1                            0   
 3             1                            0   
 4             1                            1   
 
    Компетентность работника в общение  Не удовлетворен  \
 0                                   0                0   
 1                                   0                0   
 2                                   0                0   
 3                                   0  

### Разделение данных для обучения моделей

In [39]:
X_raw = df["comment"]
y = df.iloc[:, 3:]


# Векторизация текста
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(X_raw)
X = X.toarray()
y_array = y.values.astype(int)

# Стратифицированное разбиение (70% train, 30% test)
X_train, y_train, X_test, y_test = iterative_train_test_split(
    X, y_array, test_size=0.3
)

# Преобразуем обратно в DataFrame, если нужно
X_train_raw = X_raw.iloc[np.ravel(np.where(np.isin(X, X_train).all(axis=1)))].reset_index(drop=True)
X_test_raw = X_raw.iloc[np.ravel(np.where(np.isin(X, X_test).all(axis=1)))].reset_index(drop=True)

### Logistic Regression

In [40]:
model = MultiOutputClassifier(LogisticRegression(max_iter=2000, class_weight='balanced'))
model.fit(X_train, y_train)

# Предсказания
y_pred = model.predict(X_test)

print("\nLogistic Regression")
print("{:<40} {:>10} {:>10}".format("Label", "ROC-AUC", "F1"))
print("-" * 65)
roc_auc_values = []
f1_values = []

for i, col in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test[:, i], y_pred[:, i])
        f1 = f1_score(y_test[:, i], y_pred[:, i])
        print(f"{col:<40} {auc:>10.3f} {f1:>10.3f}")
        roc_auc_values.append(auc)
        f1_values.append(f1)
    except Exception as e:
        print(f"{col:<40} {'error':>10} {'error':>10} - {str(e)}")

if roc_auc_values:
    avg_roc_auc = np.mean(roc_auc_values)
    avg_f1 = np.mean(f1_values)
    print("-" * 65)
    print(f"{'Average ROC-AUC / F1':<40} {avg_roc_auc:>10.3f} {avg_f1:>10.3f}")

joblib.dump(model, "model/logistic_model.pkl")


Logistic Regression
Label                                       ROC-AUC         F1
-----------------------------------------------------------------
Вопрос решен                                  0.849      0.953
Качество выполненной работы                   0.679      0.492
Компетентность работника в общение            0.742      0.559
Не удовлетворен                               0.848      0.646
Скорость выполненной работы                   0.900      0.886
Не понравился результат                       0.786      0.483
Не понравилось качество                       0.800      0.556
-----------------------------------------------------------------
Average ROC-AUC / F1                          0.801      0.654


['model/logistic_model.pkl']

### Random Forest

In [41]:
rf_model = MultiOutputClassifier(RandomForestClassifier(n_estimators=1000, random_state=42))
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\nRandom Forest")
print("{:<40} {:>10} {:>10}".format("Label", "ROC-AUC", "F1"))
print("-" * 65)
roc_auc_values_rf = []
f1_values_rf = []

for i, col in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test[:, i], y_pred_rf[:, i])
        f1 = f1_score(y_test[:, i], y_pred_rf[:, i])
        print(f"{col:<40} {auc:>10.3f} {f1:>10.3f}")
        roc_auc_values_rf.append(auc)
        f1_values_rf.append(f1)
    except Exception as e:
        print(f"{col:<40} {'error':>10} {'error':>10} - {str(e)}")

if roc_auc_values_rf:
    avg_roc_auc_rf = np.mean(roc_auc_values_rf)
    avg_f1_rf = np.mean(f1_values_rf)
    print("-" * 65)
    print(f"{'Average ROC-AUC / F1':<40} {avg_roc_auc_rf:>10.3f} {avg_f1_rf:>10.3f}")

joblib.dump(rf_model, "model/random_forest_model.pkl")


Random Forest
Label                                       ROC-AUC         F1
-----------------------------------------------------------------
Вопрос решен                                  0.553      0.950
Качество выполненной работы                   0.660      0.480
Компетентность работника в общение            0.579      0.270
Не удовлетворен                               0.601      0.329
Скорость выполненной работы                   0.910      0.897
Не понравился результат                       0.515      0.059
Не понравилось качество                       0.509      0.036
-----------------------------------------------------------------
Average ROC-AUC / F1                          0.618      0.432


['model/random_forest_model.pkl']

### Linear SVC

In [42]:
svc_model = OneVsRestClassifier(LinearSVC(max_iter=10000))
svc_model.fit(X_train, y_train)
y_pred_svc = svc_model.predict(X_test)

print("\nLinear SVC")
print("{:<40} {:>10} {:>10}".format("Label", "ROC-AUC", "F1"))
print("-" * 65)
roc_auc_values_svc = []
f1_values_svc = []

for i, col in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test[:, i], y_pred_svc[:, i])
        f1 = f1_score(y_test[:, i], y_pred_svc[:, i])
        print(f"{col:<40} {auc:>10.3f} {f1:>10.3f}")
        roc_auc_values_svc.append(auc)
        f1_values_svc.append(f1)
    except Exception as e:
        print(f"{col:<40} {'error':>10} {'error':>10} - {str(e)}")

if roc_auc_values_svc:
    avg_roc_auc_svc = np.mean(roc_auc_values_svc)
    avg_f1_svc = np.mean(f1_values_svc)
    print("-" * 65)
    print(f"{'Average ROC-AUC / F1':<40} {avg_roc_auc_svc:>10.3f} {avg_f1_svc:>10.3f}")

joblib.dump(svc_model, "model/linear_svc_model.pkl")


Linear SVC
Label                                       ROC-AUC         F1
-----------------------------------------------------------------
Вопрос решен                                  0.606      0.955
Качество выполненной работы                   0.677      0.517
Компетентность работника в общение            0.658      0.471
Не удовлетворен                               0.652      0.442
Скорость выполненной работы                   0.904      0.890
Не понравился результат                       0.528      0.108
Не понравилось качество                       0.560      0.212
-----------------------------------------------------------------
Average ROC-AUC / F1                          0.655      0.514


['model/linear_svc_model.pkl']

### CatBoost

In [43]:
# Настройка базовой модели CatBoost
base_model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.1,
    depth=6,
    auto_class_weights='Balanced',  # Для борьбы с дисбалансом
    verbose=0
)

# Обёртка для многоклассовой многолейбл задачи
model = MultiOutputClassifier(base_model)

# Обучение
model.fit(X_train, y_train)

# Предсказания
y_pred = model.predict(X_test)

# Оценка
print("\nCatBoostClassifier")
print("{:<40} {:>10} {:>10}".format("Label", "ROC-AUC", "F1"))
print("-" * 65)
roc_auc_values = []
f1_values = []

for i, col in enumerate(y.columns):
    try:
        auc = roc_auc_score(y_test[:, i], y_pred[:, i])
        f1 = f1_score(y_test[:, i], y_pred[:, i])
        print(f"{col:<40} {auc:>10.3f} {f1:>10.3f}")
        roc_auc_values.append(auc)
        f1_values.append(f1)
    except Exception as e:
        print(f"{col:<40} {'error':>10} {'error':>10} - {str(e)}")

if roc_auc_values:
    avg_roc_auc = np.mean(roc_auc_values)
    avg_f1 = np.mean(f1_values)
    print("-" * 65)
    print(f"{'Average ROC-AUC / F1':<40} {avg_roc_auc:>10.3f} {avg_f1:>10.3f}")

# Сохранение модели
joblib.dump(model, "model/catboost_model.pkl")


CatBoostClassifier
Label                                       ROC-AUC         F1
-----------------------------------------------------------------
Вопрос решен                                  0.751      0.953
Качество выполненной работы                   0.685      0.505
Компетентность работника в общение            0.715      0.560
Не удовлетворен                               0.743      0.560
Скорость выполненной работы                   0.928      0.916
Не понравился результат                       0.669      0.347
Не понравилось качество                       0.738      0.513
-----------------------------------------------------------------
Average ROC-AUC / F1                          0.747      0.622


['model/catboost_model.pkl']